In [1]:
from quoteprovider import get_klines_iter

get_klines_iter('BTCUSDT', '1d', '2022-11-01', '2022-12-01')

,Date,Time,Open,High,Low,Close,Volume
0,01/11/2022,00:00:00,20490.74,20700.00,20330.74,20483.62,279932.437710
1,02/11/2022,00:00:00,20482.81,20800.00,20048.04,20151.84,373716.272990
2,03/11/2022,00:00:00,20151.84,20393.32,20031.24,20207.82,319185.154400
3,04/11/2022,00:00:00,20207.12,21302.05,20180.96,21148.52,453694.391650
4,05/11/2022,00:00:00,21148.52,21480.65,21080.65,21299.37,245621.985250
5,06/11/2022,00:00:00,21299.37,21365.27,20886.13,20905.58,230036.972170
6,07/11/2022,00:00:00,20905.58,21069.77,20384.89,20591.13,386977.603370
7,08/11/2022,00:00:00,20590.67,20700.88,17166.83,18547.23,760705.362783
8,09/11/2022,00:00:00,18545.38,18587.76,15588.00,15922.81,731926.929729
9,10/11/2022,00:00:00,15922.68,18199.00,15754.26,17601.15,608448.364320


In [5]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import calendar

def get_klines_iter(symbol, interval, start, end = None, limit=1000):
    # start and end must be isoformat YYYY-MM-DD
    # We are using utc time zone

    # the maximum records is 1000 per each Binance API call

    df = pd.DataFrame()

    if start is None:
        print('start time must not be None')
        return
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000

    if end is None:
        dt = datetime.now(timezone.utc)
        utc_time = dt.replace(tzinfo=timezone.utc)
        end = int(utc_time.timestamp()) * 1000
        return
    else:
        end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000
    last_time = None

    while len(df) == 0 or (last_time is not None and last_time < end):
        url = 'https://api.binance.com/api/v3/klines?symbol=' + \
              symbol + '&interval=' + interval + '&limit=1000'
        if(len(df) == 0):
            url += '&startTime=' + str(start)
        else:
            url += '&startTime=' + str(last_time)

        url += '&endTime=' + str(end)
        df2 = pd.read_json(url)
        df2.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']
        dftmp = pd.DataFrame()
        dftmp = pd.concat([df2, dftmp], axis=0, ignore_index=True, keys=None)

        dftmp.Opentime = pd.to_datetime(dftmp.Opentime, unit='ms')
        dftmp['Date'] = dftmp.Opentime.dt.strftime("%d/%m/%Y")
        dftmp['Time'] = dftmp.Opentime.dt.strftime("%H:%M:%S")
        dftmp = dftmp.drop(['Quote asset volume', 'Closetime', 'Opentime',
                      'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore'], axis=1)
        column_names = ["Date", "Time", "Open", "High", "Low", "Close", "Volume"]
        dftmp.reset_index(drop=True, inplace=True)
        dftmp = dftmp.reindex(columns=column_names)
        string_dt = str(dftmp['Date'][len(dftmp) - 1]) + 'T' + str(dftmp['Time'][len(dftmp) - 1]) + '.000Z'
        utc_last_time = datetime.strptime(string_dt, "%d/%m/%YT%H:%M:%S.%fZ")
        last_time = (utc_last_time - datetime(1970, 1, 1)) // timedelta(milliseconds=1)
        df = pd.concat([df, dftmp], axis=0, ignore_index=True, keys=None)
    df.to_csv('0y_eth_only17andnew.csv', sep='\t', index=False)
get_klines_iter('ETHBUSD', '30m', '2022-01-01', '2022-02-21')

In [9]:
from quoteprovider import get_klines_iter

In [13]:
get_klines_iter('BTCUSDT', '1d', '2022-12-04', '2022-12-06')

,Date,Time,Open,High,Low,Close,Volume
0,04/12/2022,00:00:00,16885.20,17049.45,16878.25,16959.64,27682.51453
1,04/12/2022,04:00:00,16959.64,17068.00,16958.15,16996.39,30157.22705
2,04/12/2022,08:00:00,16996.39,17050.00,16905.01,16948.64,26725.06059
3,04/12/2022,12:00:00,16948.64,17067.45,16915.00,17029.88,33461.22340
4,04/12/2022,16:00:00,17029.88,17159.00,16972.36,17085.53,37721.27121
5,04/12/2022,20:00:00,17085.53,17202.84,17069.01,17105.70,22871.83709
6,05/12/2022,00:00:00,17106.65,17349.99,17076.85,17196.88,49755.20725
7,05/12/2022,04:00:00,17197.22,17424.25,17191.74,17313.93,43938.99382
8,05/12/2022,08:00:00,17313.34,17386.00,17281.42,17310.51,34517.04213
9,05/12/2022,12:00:00,17310.08,17313.48,17015.32,17083.92,38567.89848


In [1]:
from tradingview_ta import TA_Handler, Interval, Exchange

In [2]:
import pandas as pd
df = {}

In [3]:
def recomendation(interval):
    pairs = ['EURSGD','GBPSGD','USDSGD','EURPLN','GBPPLN','USDPLN', 'USDMXN']
    df = {}
    for pair in pairs:

        output = TA_Handler(
            symbol=pair,
            screener='forex',
            exchange='FX_IDC',
            interval=interval
        )
        df[pair] = output.get_analysis().summary['RECOMMENDATION']
    df['index'] = output.get_analysis().time
    recom = pd.DataFrame([df])

    recom.set_index('index', inplace=True)
    return recom

In [4]:
#Daily
recomendation(Interval.INTERVAL_1_DAY)

Exception: Exchange or symbol not found.

In [5]:
recomendation(Interval.INTERVAL_4_HOURS)

,EURSGD,GBPSGD,USDSGD,EURPLN,GBPPLN,USDPLN,USDMXN
index,,,,,,,
2022-12-27 17:12:33.908303,SELL,SELL,SELL,BUY,NEUTRAL,BUY,SELL


In [5]:
from tradingview_ta import *
analysis = get_multiple_analysis(screener="forex", interval=Interval.INTERVAL_1_HOUR, symbols=["FX_IDC:EURPLN",'FX_IDC:EURSGD','FX_IDC:EURUSD','FX_IDC:GBPPLN','FX_IDC:GBPSGD','FX_IDC:USDMXN','FX_IDC:USDPLN','FX_IDC:USDSGD','FX_IDC:USDZAR'])

In [6]:
for i in analysis.values():
    print(i.summary['RECOMMENDATION'])

BUY
STRONG_SELL
BUY
NEUTRAL
BUY
STRONG_SELL
SELL
BUY
SELL


In [7]:
output = TA_Handler(
    symbol='BTCUSDTPERP',
    screener='Crypto',
    exchange='BINANCE',
    interval=Interval.INTERVAL_1_DAY
)

In [9]:
output.get_analysis().summary['RECOMMENDATION']

'SELL'